In [ ]:
import pandas as pd
import sys
import os
import glob

%load_ext cuebiqmagic.magics
%init_cuebiq_data 
snow_engine = get_ipython().user_ns['instance']

In [ ]:
engine = getattr(snow_engine, "engine", None) \
      or getattr(snow_engine, "connection", None)

In [ ]:
plinio_schema = 'DEDICATED.PLINIO_LA_FIRES_EXPORTS'

In [ ]:
ddl = f"CREATE SCHEMA IF NOT EXISTS {plinio_schema}"
snow_engine.execute_statement(ddl)

In [ ]:
table_names = []
for path in glob.glob("./od_matrices/*.csv"):
    table_name = os.path.splitext(os.path.basename(path))[0].replace("=", "_")
    table_names.append(table_name)
    df = pd.read_csv(path)
    snow_engine.write_dataframe(df, table_name, schema=plinio_schema)

## Move to export stage

In [ ]:
q = '''SELECT table_name
FROM   DEDICATED.INFORMATION_SCHEMA.TABLES
WHERE  table_schema = 'PLINIO_LA_FIRES_EXPORTS'
  AND  table_type   = 'BASE TABLE'
ORDER  BY table_name
'''
all_tables = snow_engine.read_sql(q)

In [ ]:
all_tables.table_name

In [ ]:
export_stage = '@dedicated.spectus_s3_export.spectus_s3_export/la-fires/'
origin_schema = 'DEDICATED.PLINIO_LA_FIRES_EXPORTS'

In [ ]:
for table in all_tables.table_name:
    orig = origin_schema + '."' + table + '"'
    dest = export_stage + table + '/'
    export_q = f'''
    COPY INTO {dest}
    FROM {orig}
    FILE_FORMAT = (TYPE = PARQUET  COMPRESSION = SNAPPY)
    MAX_FILE_SIZE = 5000000000;
    '''
    print("executing query: \n", export_q)
    snow_engine.execute_statement(export_q)
    print("executed...")